In [1]:
import pandas as pd
import requests
import matplotlib
import pymongo
import numpy as np
import re
import pickle as pkl
import json

In [2]:
MONGO_CONNECT_URL="mongodb://localhost:27017/IR_Project"
db_name = "ir_535"
client = pymongo.MongoClient(MONGO_CONNECT_URL)
db = client.get_database(db_name)

POI LIST

In [33]:
poi_tweets = list(db['tweets'].find({
    "poi_name": {"$exists": True}
}))
poi_names_all = [tweet['poi_name'] for tweet in poi_tweets]
poi_names = list(np.unique(poi_names_all))
poi_names

['ArvindKejriwal',
 'BernieSanders',
 'CDCgov',
 'Claudiashein',
 'EnriqueAlfaroR',
 'GovRonDeSantis',
 'GregAbbott_TX',
 'HHS_ASH',
 'HLGatell',
 'JoeBiden',
 'KamalaHarris',
 'LindseyGrahamSC',
 'MamataOfficial',
 'MoHFW_INDIA',
 'POTUS',
 'PeteButtigieg',
 'RahulGandhi',
 'RandPaul',
 'RicardoMonrealA',
 'SSalud_mx',
 'SecBecerra',
 'SenSanders',
 'VP',
 'VicenteFoxQue',
 'alfredodelmazo',
 'alitomorenoc',
 'amyklobuchar',
 'fernandeznorona',
 'lopezobrador_',
 'm_ebrard',
 'mansukhmandviya',
 'marcorubio',
 'mattgaetz',
 'mtgreenee',
 'narendramodi',
 'sambitswaraj',
 'tatclouthier',
 'tedcruz']

Find Sentiment of tweets across the entire corpus

In [34]:

data_entire_corpus = {
    "POSITIVE" :  32574,
    "NEGATIVE" : 54124,
    "NEUTRAL" :  135416,
    "MIXED" : 8996
}

In [35]:
# FOR COUNTRY based individual sentiments.
country_based_sentiments = {}
for country in ["USA", "India", "Mexico"]:
    country_based_sentiments[country] = {}
    for sentiment in ["POSITIVE", "NEGATIVE", "NEUTRAL", "MIXED"]:
        country_based_sentiments[country][sentiment] = {}
        out = list(db['tweets'].aggregate([
            {
                '$match': {
                    'country': re.compile(r""+country+r"(?i)"), 
                    'sentiment': sentiment
                }
            }, {
                '$count': 'sentiment'
            }
        ]))
        country_based_sentiments[country][sentiment] = out[0]['sentiment']

/var/folders/d_/5fj9_qbs35qb24skz6z1nkpw0000gn/T/ipykernel_2944/1974778830.py:10: DeprecationWarning: Flags not at the start of the expression 'USA(?i)'
  'country': re.compile(r""+country+r"(?i)"),
/var/folders/d_/5fj9_qbs35qb24skz6z1nkpw0000gn/T/ipykernel_2944/1974778830.py:10: DeprecationWarning: Flags not at the start of the expression 'India(?i)'
  'country': re.compile(r""+country+r"(?i)"),
/var/folders/d_/5fj9_qbs35qb24skz6z1nkpw0000gn/T/ipykernel_2944/1974778830.py:10: DeprecationWarning: Flags not at the start of the expression 'Mexico(?i)'
  'country': re.compile(r""+country+r"(?i)"),


In [36]:
country_based_sentiments

{'USA': {'POSITIVE': 17468,
  'NEGATIVE': 40110,
  'NEUTRAL': 78148,
  'MIXED': 2966},
 'India': {'POSITIVE': 8052, 'NEGATIVE': 4899, 'NEUTRAL': 28782, 'MIXED': 342},
 'Mexico': {'POSITIVE': 6957,
  'NEGATIVE': 8886,
  'NEUTRAL': 28124,
  'MIXED': 5668}}

In [37]:
# Calculated DATA 
country_based_sentiments = {
    'USA': {
        'POSITIVE': 17468,
        'NEGATIVE': 40110,
        'NEUTRAL': 78148,
        'MIXED': 2966
    },
    'India': {
        'POSITIVE': 8052,
        'NEGATIVE': 4899,
        'NEUTRAL': 28782,
        'MIXED': 342
    },
    'Mexico': {
        'POSITIVE': 6957,
        'NEGATIVE': 8886,
        'NEUTRAL': 28124,
        'MIXED': 5668
    }
}

# POI DATA

In [38]:
poi_tweet_based_sentiments = {}
for poi in poi_names:
    poi_tweet_based_sentiments[poi] = {}
    for sentiment in ["POSITIVE", "NEGATIVE", "NEUTRAL", "MIXED"]:
        poi_tweet_based_sentiments[poi][sentiment] = {}
        out = list(db['tweets'].aggregate([
            {
                '$match': {
                    'poi_name': poi, 
                    'sentiment': sentiment
                }
            }, {
                '$count': 'sentiment'
            }
        ]))
        if len(out) != 0:
            poi_tweet_based_sentiments[poi][sentiment] = out[0]['sentiment']
        else:
            print(f"poi_name: {poi} || sentiment : {sentiment}")
            poi_tweet_based_sentiments[poi][sentiment] = 0

poi_name: MoHFW_INDIA || sentiment : MIXED


In [40]:
poi_tweet_based_sentiments = {
    'ArvindKejriwal': {
        'POSITIVE': 549,
        'NEGATIVE': 168,
        'NEUTRAL': 708,
        'MIXED': 15
    },
    'BernieSanders': {
        'POSITIVE': 164,
        'NEGATIVE': 490,
        'NEUTRAL': 663,
        'MIXED': 79
    },
    'CDCgov': {
        'POSITIVE': 70,
        'NEGATIVE': 58,
        'NEUTRAL': 1693,
        'MIXED': 14
    },
    'Claudiashein': {
        'POSITIVE': 242,
        'NEGATIVE': 6,
        'NEUTRAL': 439,
        'MIXED': 32
    },
    'EnriqueAlfaroR': {
        'POSITIVE': 162,
        'NEGATIVE': 36,
        'NEUTRAL': 406,
        'MIXED': 53
    },
    'GovRonDeSantis': {
        'POSITIVE': 398,
        'NEGATIVE': 66,
        'NEUTRAL': 663,
        'MIXED': 8
    },
    'GregAbbott_TX': {
        'POSITIVE': 348,
        'NEGATIVE': 255,
        'NEUTRAL': 621,
        'MIXED': 10
    },
    'HHS_ASH': {
        'POSITIVE': 181,
        'NEGATIVE': 34,
        'NEUTRAL': 559,
        'MIXED': 6
    },
    'HLGatell': {
        'POSITIVE': 48,
        'NEGATIVE': 15,
        'NEUTRAL': 724,
        'MIXED': 6
    },
    'JoeBiden': {
        'POSITIVE': 334,
        'NEGATIVE': 251,
        'NEUTRAL': 925,
        'MIXED': 39
    },
    'KamalaHarris': {
        'POSITIVE': 399,
        'NEGATIVE': 238,
        'NEUTRAL': 889,
        'MIXED': 17
    },
    'LindseyGrahamSC': {
        'POSITIVE': 193,
        'NEGATIVE': 406,
        'NEUTRAL': 528,
        'MIXED': 47
    },
    'MamataOfficial': {
        'POSITIVE': 376,
        'NEGATIVE': 98,
        'NEUTRAL': 242,
        'MIXED': 17
    },
    'MoHFW_INDIA': {
        'POSITIVE': 185,
        'NEGATIVE': 50,
        'NEUTRAL': 1687,
        'MIXED': 0
    },
    'POTUS': {
        'POSITIVE': 262,
        'NEGATIVE': 92,
        'NEUTRAL': 566,
        'MIXED': 40
    },
    'PeteButtigieg': {
        'POSITIVE': 317,
        'NEGATIVE': 205,
        'NEUTRAL': 452,
        'MIXED': 15
    },
    'RahulGandhi': {
        'POSITIVE': 327,
        'NEGATIVE': 556,
        'NEUTRAL': 520,
        'MIXED': 33
    },
    'RandPaul': {
        'POSITIVE': 207,
        'NEGATIVE': 344,
        'NEUTRAL': 556,
        'MIXED': 19
    },
    'RicardoMonrealA': {
        'POSITIVE': 145,
        'NEGATIVE': 36,
        'NEUTRAL': 759,
        'MIXED': 55
    },
    'SSalud_mx': {
        'POSITIVE': 570,
        'NEGATIVE': 11,
        'NEUTRAL': 1670,
        'MIXED': 241
    },
    'SecBecerra': {
        'POSITIVE': 175,
        'NEGATIVE': 25,
        'NEUTRAL': 232,
        'MIXED': 10
    },
    'SenSanders': {
        'POSITIVE': 90,
        'NEGATIVE': 411,
        'NEUTRAL': 348,
        'MIXED': 60
    },
    'VP': {
        'POSITIVE': 272,
        'NEGATIVE': 47,
        'NEUTRAL': 411,
        'MIXED': 16
    },
    'VicenteFoxQue': {
        'POSITIVE': 243,
        'NEGATIVE': 529,
        'NEUTRAL': 339,
        'MIXED': 236
    },
    'alfredodelmazo': {
        'POSITIVE': 244,
        'NEGATIVE': 4,
        'NEUTRAL': 1407,
        'MIXED': 8
    },
    'alitomorenoc': {
        'POSITIVE': 761,
        'NEGATIVE': 93,
        'NEUTRAL': 296,
        'MIXED': 67
    },
    'amyklobuchar': {
        'POSITIVE': 509,
        'NEGATIVE': 241,
        'NEUTRAL': 660,
        'MIXED': 52
    },
    'fernandeznorona': {
        'POSITIVE': 315,
        'NEGATIVE': 383,
        'NEUTRAL': 781,
        'MIXED': 207
    },
    'lopezobrador_': {
        'POSITIVE': 274,
        'NEGATIVE': 75,
        'NEUTRAL': 1126,
        'MIXED': 105
    },
    'm_ebrard': {
        'POSITIVE': 450,
        'NEGATIVE': 76,
        'NEUTRAL': 999,
        'MIXED': 88
    },
    'mansukhmandviya': {
        'POSITIVE': 1420,
        'NEGATIVE': 87,
        'NEUTRAL': 1640,
        'MIXED': 3
    },
    'marcorubio': {
        'POSITIVE': 122,
        'NEGATIVE': 350,
        'NEUTRAL': 552,
        'MIXED': 28
    },
    'mattgaetz': {
        'POSITIVE': 210,
        'NEGATIVE': 367,
        'NEUTRAL': 584,
        'MIXED': 8
    },
    'mtgreenee': {
        'POSITIVE': 214,
        'NEGATIVE': 886,
        'NEUTRAL': 387,
        'MIXED': 33
    },
    'narendramodi': {
        'POSITIVE': 872,
        'NEGATIVE': 55,
        'NEUTRAL': 929,
        'MIXED': 19
    },
    'sambitswaraj': {
        'POSITIVE': 394,
        'NEGATIVE': 82,
        'NEUTRAL': 1171,
        'MIXED': 5
    },
    'tatclouthier': {
        'POSITIVE': 166,
        'NEGATIVE': 33,
        'NEUTRAL': 382,
        'MIXED': 22
    },
    'tedcruz': {
        'POSITIVE': 163,
        'NEGATIVE': 591,
        'NEUTRAL': 614,
        'MIXED': 20
    }
}

In [ ]:
{'USA': ['BernieSanders',
  'CDCgov',
  'GovRonDeSantis',
  'GregAbbott_TX',
  'HHS_ASH',
  'JoeBiden',
  'KamalaHarris',
  'LindseyGrahamSC',
  'POTUS',
  'RandPaul',
  'SecBecerra',
  'SenSanders',
  'VP',
  'VicenteFoxQue',
  'amyklobuchar',
  'fernandeznorona',
  'mattgaetz'],
 'INDIA': ['ArvindKejriwal',
  'MamataOfficial',
  'MoHFW_INDIA',
  'RahulGandhi',
  'mansukhmandviya',
  'narendramodi',
  'sambitswaraj'],
 'MEXICO': ['Claudiashein',
  'EnriqueAlfaroR',
  'HLGatell',
  'PeteButtigieg',
  'RicardoMonrealA',
  'SSalud_mx',
  'alfredodelmazo',
  'alitomorenoc',
  'lopezobrador_',
  'm_ebrard',
  'marcorubio',
  'mtgreenee',
  'tatclouthier',
  'tedcruz']}

In [24]:
# Sentiments of tweets overtime .....

In [6]:
poi_country = [
    ['ArvindKejriwal', 'INDIA'],
    ['BernieSanders', 'USA'],
    ['CDCgov', 'USA'],
    ['Claudiashein', 'MEXICO'],
    ['EnriqueAlfaroR', 'MEXICO'],
    ['GovRonDeSantis', 'USA'],
    ['GregAbbott_TX', 'USA'],
    ['HHS_ASH', 'USA'],
    ['HLGatell', 'MEXICO'],
    ['JoeBiden', 'USA'],
    ['KamalaHarris', 'USA'],
    ['LindseyGrahamSC', 'USA'],
    ['MamataOfficial', 'INDIA'],
    ['MoHFW_INDIA', 'INDIA'],
    ['POTUS', 'USA'],
    ['PeteButtigieg', 'MEXICO'],
    ['RahulGandhi', 'INDIA'],
    ['RandPaul', 'USA'],
    ['RicardoMonrealA', 'MEXICO'],
    ['SSalud_mx', 'MEXICO'],
    ['SecBecerra', 'USA'],
    ['SenSanders', 'USA'],
    ['VP', 'USA'],
    ['VicenteFoxQue', 'USA'],
    ['alfredodelmazo', 'MEXICO'],
    ['alitomorenoc', 'MEXICO'],
    ['amyklobuchar', 'USA'],
    ['fernandeznorona', 'USA'],
    ['lopezobrador_', 'MEXICO'],
    ['m_ebrard', 'MEXICO'],
    ['mansukhmandviya', 'INDIA'],
    ['marcorubio', 'MEXICO'],
    ['mattgaetz', 'USA'],
    ['mtgreenee', 'MEXICO'],
    ['narendramodi', 'INDIA'],
    ['sambitswaraj', 'INDIA'],
    ['tatclouthier', 'MEXICO'],
    ['tedcruz', 'MEXICO']
]

In [41]:
countryWise = {
    "USA": [],
    "INDIA": [],
    "MEXICO": [],
}
for item in poi_country:
    countryWise[item[1]].append(item[0])

NameError: name 'poi_country' is not defined

In [4]:
# countryWise

In [42]:
response = list(db['tweets'].aggregate([
    {
        '$match': {
            'hashtags': {
                '$exists': True
            }
        }
    }
]))

In [43]:
hashtags_list_all = [item["hashtags"] for item in response]

In [44]:
hashtags_flat = [item for sublist in hashtags_list_all for item in sublist]
hashtags_li , hashtags_count = np.unique(hashtags_flat, return_counts=True)

In [56]:
hashtags_li = hashtags_li.tolist()
hashtags_count = hashtags_count.tolist()

In [57]:
hashtags_word_colud_corpus = []
for i in range(0, len(hashtags_count)):
    hashtags_word_colud_corpus.append({
        "name": hashtags_li[i],
        "weight": hashtags_count[i],
    })

In [58]:
hashtags_word_colud_corpus

[{'name': '035X_A', 'weight': 1},
 {'name': '1000am', 'weight': 1},
 {'name': '100ArgentinosDicen', 'weight': 2},
 {'name': '100AsianHeartthrobs2021', 'weight': 1},
 {'name': '100Años', 'weight': 3},
 {'name': '100AñosReyesHeroles', 'weight': 1},
 {'name': '100Crore', 'weight': 1},
 {'name': '100DayCountdown', 'weight': 6},
 {'name': '100DayCountdownAlarm', 'weight': 2},
 {'name': '100DaysAsAFrontendDev', 'weight': 1},
 {'name': '100DaysNoVikas', 'weight': 1},
 {'name': '100DaysOfCode', 'weight': 5},
 {'name': '100DaysToGo', 'weight': 2},
 {'name': '100MostHandsomeMen2021', 'weight': 2},
 {'name': '100crorevaccinations', 'weight': 3},
 {'name': '100daycountdown', 'weight': 1},
 {'name': '100daysofcode', 'weight': 1},
 {'name': '100daysofcodewithGFG', 'weight': 1},
 {'name': '100daysofwalking', 'weight': 1},
 {'name': '100facesSCHA', 'weight': 1},
 {'name': '100fact', 'weight': 1},
 {'name': '100million', 'weight': 1},
 {'name': '100millones', 'weight': 3},
 {'name': '100percenthays', '

In [59]:
data = {
    "poi_names": poi_names,
    "across_entire_corpus": data_entire_corpus,
    "country_based_sentiments": country_based_sentiments,
    "poi_tweet_based_sentiments": poi_tweet_based_sentiments,
    "hashtags_word_colud_corpus": hashtags_word_colud_corpus
}
f = open("sentiment_corpus.pkl", "wb")
pkl.dump(data, f, pkl.HIGHEST_PROTOCOL)
f.close()

In [60]:
f = open("sentiment_corpus.json", "w")
json.dump(data, f)
f.close()